In [1]:
import os
import pandas as pd
import re
import numpy as np
from tqdm import tqdm
from ioh_xplainer import explainer
import pandas as pd

from ConfigSpace import ConfigurationSpace
from ConfigSpace.util import generate_grid
from IPython.display import display

# read first data file
#data_file = "mod_de_all.pkl"
#df1 = pd.read_pickle(data_file)
#df1['CR'] = 0.5
#df1['F'] = 0.5
#df1['lambda_'] = 'nan' #4 + np.floor(3 * np.log(self.d)) 6 for d=5, 8 for d=30

df2 = pd.read_pickle("intermediate-de-1.pkl")
df3 = pd.read_pickle("intermediate-de-2.pkl")
df4 = pd.read_csv("intermediate-de-3.csv")
df4 = pd.read_csv("intermediate-de-4.csv")
df4 = pd.read_csv("intermediate-de-5.csv")

dfall = pd.concat([ df2, df3, df4], ignore_index=True)

dfall = dfall.drop(columns=['Unnamed: 0'])  
print(dfall.describe())


#replacing stuff to fix
dfall['mutation_reference'] = dfall['mutation_reference'].replace(np.nan,'nan')
dfall['adaptation_method'] = dfall['adaptation_method'].replace(np.nan,'nan')
dfall['lambda_'] = dfall['lambda_'].replace(np.nan,'nan')
dfall['lambda_'] = dfall['lambda_'].replace('2', 2.0)
dfall['lambda_'] = dfall['lambda_'].replace('10', 10.0)

dfall.loc[(dfall['lambda_'] == 10.0) & (dfall['dim'] == 30),'lambda_'] = 300
dfall.loc[(dfall['lambda_'] == 10.0) & (dfall['dim'] == 5),'lambda_'] = 150
dfall.loc[(dfall['lambda_'] == 2.0) & (dfall['dim'] == 30),'lambda_'] = 60
dfall.loc[(dfall['lambda_'] == 2.0) & (dfall['dim'] == 5),'lambda_'] = 10
dfall.loc[(dfall['lambda_'] == 'nan') & (dfall['dim'] == 5),'lambda_'] = 6
dfall.loc[(dfall['lambda_'] == 'nan') & (dfall['dim'] == 30),'lambda_'] = 8

dfall.to_pickle("df_huge.pkl")
print(dfall['lambda_'].describe())

/home/bas/repos/ioh-explainer/env/lib/python3.8/site-packages/numpy/core/getlimits.py:518: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/bas/repos/ioh-explainer/env/lib/python3.8/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/bas/repos/ioh-explainer/env/lib/python3.8/site-packages/numpy/core/getlimits.py:518: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/bas/repos/ioh-explainer/env/lib/python3.8/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
IProgress not found. Please update jupyter and ipywidgets. See

                fid           iid           dim          seed            CR  \
count  1.140509e+07  1.140509e+07  1.140509e+07  1.140509e+07  1.140509e+07   
mean   1.250000e+01  3.000000e+00  1.750000e+01  1.000000e+00  4.096450e-01   
std    6.922187e+00  1.705614e+00  1.250000e+01  8.164966e-01  3.281464e-01   
min    1.000000e+00  1.000000e+00  5.000000e+00  0.000000e+00  5.000000e-02   
25%    6.750000e+00  1.000000e+00  5.000000e+00  0.000000e+00  5.000000e-02   
50%    1.250000e+01  3.000000e+00  1.750000e+01  1.000000e+00  2.500000e-01   
75%    1.825000e+01  5.000000e+00  3.000000e+01  2.000000e+00  7.500000e-01   
max    2.400000e+01  5.000000e+00  3.000000e+01  2.000000e+00  1.000000e+00   

                  F  mutation_n_comps           auc  
count  1.140509e+07      1.140509e+07  1.140509e+07  
mean   6.881960e-01      1.499987e+00  5.068084e+03  
std    4.202816e-01      5.000000e-01  3.050123e+03  
min    2.500000e-01      1.000000e+00  1.000000e+00  
25%    5.000000e-0

In [2]:
dfall.columns

Index(['fid', 'iid', 'dim', 'seed', 'CR', 'F', 'adaptation_method',
       'crossover', 'lambda_', 'lpsr', 'mutation_base', 'mutation_n_comps',
       'mutation_reference', 'use_archive', 'auc'],
      dtype='object')

In [3]:
import os
import pandas as pd
import re
import numpy as np
from tqdm import tqdm
from ioh_xplainer import explainer
import pandas as pd

from ConfigSpace import ConfigurationSpace
from ConfigSpace.util import generate_grid
from IPython.display import display

data_file = "df_huge.pkl"
df = pd.read_pickle(data_file)

features= ['mutation_base', 'mutation_reference', 'CR', "F", 'lambda_',
       'mutation_n_comps', 'use_archive', 'crossover', 'adaptation_method',
       'lpsr']

config_dict = {}
for f in features:
    config_dict[f] = list(map(str, df[f].unique()))

config_dict['mutation_n_comps'] = [1,2]
config_dict['use_archive'] = [False, True]
config_dict['lpsr'] = [False, True]

print(config_dict)
print( df['dim'].unique())
print( df['seed'].unique())
print( df['iid'].unique())

{'mutation_base': ['target', 'best', 'rand'], 'mutation_reference': ['pbest', 'rand', 'nan', 'best'], 'CR': ['0.05', '0.25', '0.75', '0.5', '1.0'], 'F': ['0.25', '0.75', '1.25', '1.75', '0.5'], 'lambda_': ['6', '8', '10', '60', '150', '300'], 'mutation_n_comps': [1, 2], 'use_archive': [False, True], 'crossover': ['exp', 'bin'], 'adaptation_method': ['nan', 'jDE', 'shade'], 'lpsr': [False, True]}
[ 5 30]
[0 1 2]
[1 5 2 3 4]


In [4]:


cs = ConfigurationSpace(config_dict)

print(cs)

de_explainer = explainer(None, 
                 cs , 
                 algname="mod-DE",
                 dims = [5,30],#, 10, 20, 40 
                 fids = np.arange(1,25), #,5
                 iids = len( df['iid'].unique()), #20 
                 reps = len( df['seed'].unique()), 
                 sampling_method = "grid",  #or random
                 grid_steps_dict = {},
                 sample_size = None,  #only used with random method
                 budget = 10000, #10000
                 seed = 1,
                 verbose = True)


de_explainer.load_results(data_file)
#for feature in features:
#    cmaes_explainer.df[feature] = cmaes_explainer.df[feature].astype("category")
df = de_explainer.performance_stats()
display(df.style.bar(cmap='viridis'))

#cmaes_explainer.plot(partial_dependence=False, best_config=False)
#print(de_explainer.stats[5])
#df = de_explainer.stats[5]
#print(df.to_latex(index=False,
#                  float_format="{:.2f}".format,
#                  multicolumn_format = "c"
#))  


Configuration space object:
  Hyperparameters:
    CR, Type: Categorical, Choices: {0.05, 0.25, 0.75, 0.5, 1.0}, Default: 0.05
    F, Type: Categorical, Choices: {0.25, 0.75, 1.25, 1.75, 0.5}, Default: 0.25
    adaptation_method, Type: Categorical, Choices: {nan, jDE, shade}, Default: nan
    crossover, Type: Categorical, Choices: {exp, bin}, Default: exp
    lambda_, Type: Categorical, Choices: {6, 8, 10, 60, 150, 300}, Default: 6
    lpsr, Type: Categorical, Choices: {False, True}, Default: False
    mutation_base, Type: Categorical, Choices: {target, best, rand}, Default: target
    mutation_n_comps, Type: Categorical, Choices: {1, 2}, Default: 1
    mutation_reference, Type: Categorical, Choices: {pbest, rand, nan, best}, Default: pbest
    use_archive, Type: Categorical, Choices: {False, True}, Default: False



In [5]:
from IPython.display import display

#df = de_explainer.behaviour_stats()
#df.style.bar(cmap='viridis')

#do it for f1 and f5

df = de_explainer.behaviour_stats(per_fid=False)
display(df.style.bar(cmap='viridis'))

#df = cmaes_explainer.behaviour_stats(per_fid=True)
#display(df.style.bar(cmap='viridis'))



In [6]:
de_explainer.to_latex_report(filename="mod_de_new", img_dir="de_images_new/")

In [7]:
categorical_columns = de_explainer.df.dtypes[de_explainer.df.dtypes == 'object'].index.to_list()
categorical_columns

df = de_explainer.df
for cat in categorical_columns:
    print(cat)
    df[cat] = df[cat].astype('category')
    print(df[cat].unique())
    print(df[cat].cat.codes.unique())

mutation_base
['target', 'best', 'rand']
Categories (3, object): ['best', 'rand', 'target']
[2 0 1]
mutation_reference
['pbest', 'rand', 'nan', 'best']
Categories (4, object): ['best', 'nan', 'pbest', 'rand']
[2 3 1 0]
crossover
['exp', 'bin']
Categories (2, object): ['bin', 'exp']
[1 0]
adaptation_method
['nan', 'jDE', 'shade']
Categories (3, object): ['jDE', 'nan', 'shade']
[1 0 2]
lambda_
[8, 6, 10, 60, 150, 300]
Categories (6, int64): [6, 8, 10, 60, 150, 300]
[1 0 2 3 4 5]


In [8]:
import scipy.stats as stats
import ioh
def intersection(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    return lst3
def compare(alg1, alg2, normalize=True):
    #assuming both alg1 and alg2 are explainer objects
    if not isinstance(alg1,explainer):
        raise "instance alg1 should be an explainer object"
    df1 = alg1.df
    df2 = alg2.df

    comparison_stats = {}
        
    for dim in intersection(alg1.dims, alg2.dims):
        dim_df1 = df1[df1['dim'] == dim]
        dim_df2 = df2[df2['dim'] == dim]
        stat_index = f"d={dim}"

        comparison_stats[stat_index] = pd.DataFrame(columns=["Function", 
            f"single-best {alg1.algname}", 
            f"single-best {alg2.algname}", 
            f"avg-best {alg1.algname}", 
            f"avg-best {alg2.algname}", 
            f"{alg1.algname}", 
            f"{alg2.algname}"])

        #split df per function
        #get avg best config
        _, df_best_mean1 = alg1._get_average_best(dim_df1)
        _, df_best_mean2 = alg2._get_average_best(dim_df2)

        for fid in intersection(alg1.fids, alg2.fids):
            func = ioh.get_problem(fid, dimension=dim, instance=1)
            fid_df1 = dim_df1[dim_df1['fid'] == fid]
            fid_df2 = dim_df2[dim_df2['fid'] == fid]

            _, df_single_best1 = alg1.get_single_best(fid, dim)
            _, df_single_best2 = alg2.get_single_best(fid, dim)
            
            # Define the new row to be added
            avg_best_avg1 = df_best_mean1[df_best_mean1['fid'] == fid]['auc'].mean()
            avg_best_var1 = df_best_mean1[df_best_mean1['fid'] == fid]['auc'].std()

            avg_best_avg2 = df_best_mean2[df_best_mean2['fid'] == fid]['auc'].mean()
            avg_best_var2 = df_best_mean2[df_best_mean2['fid'] == fid]['auc'].std()

            avg_avg1 = fid_df1['auc'].mean()
            avg_var1 = fid_df1['auc'].std()

            avg_avg2 = fid_df2['auc'].mean()
            avg_var2 = fid_df2['auc'].std()

            avg_single1 = df_single_best1['auc'].mean()
            var_single1 = df_single_best1['auc'].std()

            avg_single2 = df_single_best2['auc'].mean()
            var_single2 = df_single_best2['auc'].std()
            if (normalize):
                avg_single1 = avg_single1 / alg1.budget
                var_single1 = var_single1 / alg1.budget
                avg_best_avg1 = avg_best_avg1 / alg1.budget
                avg_best_var1 = avg_best_var1 / alg1.budget
                avg_avg1 = avg_avg1 / alg1.budget
                avg_var1 = avg_var1 / alg1.budget

                avg_single2 = avg_single2 / alg2.budget
                var_single2 = var_single2 / alg2.budget
                avg_best_avg2 = avg_best_avg2 / alg2.budget
                avg_best_var2 = avg_best_var2 / alg2.budget
                avg_avg2 = avg_avg2 / alg2.budget
                avg_var2 = avg_var2 / alg2.budget


            single_best1 = f"{avg_single1:.2f} ({var_single1:.2f})"
            single_best2 = f"{avg_single2:.2f} ({var_single2:.2f})"

            avg_best1 = f"{avg_best_avg1:.2f} ({avg_best_var1:.2f})"
            avg_best2 =f"{avg_best_avg2:.2f} ({avg_best_var2:.2f})"

            avg1 = f"{avg_avg1:.2f} ({avg_var1:.2f})"
            avg2 = f"{avg_avg2:.2f} ({avg_var2:.2f})"

            #single best significance
            single_sig = False
            avg_sig = False
            res = stats.ttest_ind(df_single_best1['auc'].values, df_single_best2['auc'].values)

            if res.pvalue < 0.05:
                single_sig = True
                if (avg_single1 > avg_single2):
                    single_best1 = "\\textbf{"+single_best1+"}"
                else:
                    single_best2 = "\\textbf{"+single_best2+"}"
            
            #avg best significance
            res = stats.ttest_ind(fid_df1['auc'].values, fid_df2['auc'].values)
            if res.pvalue < 0.05:
                avg_sig = True
                if (avg_best1 > avg_best2):
                    avg_best1 = "\\textbf{"+avg_best1+"}"
                else:
                    avg_best2 = "\\textbf{"+avg_best2+"}"

            #avg significance
            res = stats.ttest_ind(dim_df1['auc'].values, dim_df2['auc'].values)
            if res.pvalue < 0.05:
                avg_sig = True
                if (avg1 > avg2):
                    avg1 = "\\textbf{"+avg1+"}"
                else:
                    avg2 = "\\textbf{"+avg2+"}"


            new_row = {"Function": f"f{fid} {func.meta_data.name}", 
                    f"single-best {alg1.algname}": single_best1, 
                    f"single-best {alg2.algname}": single_best2, 
                    f"avg-best {alg1.algname}": avg_best1, 
                    f"avg-best {alg2.algname}": avg_best2, 
                    f"{alg1.algname}": avg1, 
                    f"{alg2.algname}": avg2
                }
            
            # Use the loc method to add the new row to the DataFrame
            comparison_stats[stat_index].loc[len(comparison_stats[stat_index])] = new_row
            #check if the single best is significantly better than the avg best
            
    return comparison_stats

In [9]:
#compare de with cma
data_file = "cma_es.pkl"
features = ['elitist', 'mirrored', 'base_sampler', 'weights_option', 'local_restart', 'step_size_adaptation']
df = pd.read_pickle(data_file)



cs = ConfigurationSpace({
    "elitist": list(map(str, df['elitist'].unique())), 
    "mirrored" : list(map(str, df['mirrored'].unique())), 
    "base_sampler" : list(map(str, df['base_sampler'].unique())), 
    "weights_option" : list(map(str, df['weights_option'].unique())), 
    "local_restart" : list(map(str, df['local_restart'].unique())), 
    "step_size_adaptation" : list(map(str, df['step_size_adaptation'].unique()))
})

print(cs)
cmaes_explainer = explainer(None, 
                 cs , 
                 algname="mod-CMA",
                 dims = [5,30],#, 10, 20, 40 
                 fids = np.arange(1,25), #,5
                 iids = len( df['iid'].unique()), #20 
                 reps = len( df['seed'].unique()), 
                 sampling_method = "grid",  #or random
                 grid_steps_dict = {},
                 sample_size = None,  #only used with random method
                 budget = 10000, #10000
                 seed = 1,
                 verbose = True)


cmaes_explainer.load_results(data_file)

dffinal = compare(de_explainer, cmaes_explainer)

with open(f'compare-new.tex', "w") as fh:
    for dim in ['d=5', 'd=30']:
        dffinal[dim].to_latex(buf=fh, index=False, 
                    multicolumn_format = "c", 
                    float_format="%.2f",
                    caption = f"Performance comparison of Modular CMA and Modular DE with {dim}. Boldface indicates a significant improvement either between single-best configurations, average best configurations or all configurations.")

Configuration space object:
  Hyperparameters:
    base_sampler, Type: Categorical, Choices: {sobol, gaussian, halton}, Default: sobol
    elitist, Type: Categorical, Choices: {True, False}, Default: True
    local_restart, Type: Categorical, Choices: {nan, IPOP, BIPOP}, Default: nan
    mirrored, Type: Categorical, Choices: {mirrored pairwise, nan, mirrored}, Default: mirrored pairwise
    step_size_adaptation, Type: Categorical, Choices: {csa, psr}, Default: csa
    weights_option, Type: Categorical, Choices: {default, equal, 1/2^lambda}, Default: default



Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.


In [15]:
import seaborn as sns
import matplotlib.pyplot as plt


#also load cma
data_file = "cma_es.pkl"
features2 = ['elitist', 'mirrored', 'base_sampler', 'weights_option', 'local_restart', 'step_size_adaptation']
df_temp = pd.read_pickle(data_file)



cs = ConfigurationSpace({
    "elitist": list(map(str, df_temp['elitist'].unique())), 
    "mirrored" : list(map(str, df_temp['mirrored'].unique())), 
    "base_sampler" : list(map(str, df_temp['base_sampler'].unique())), 
    "weights_option" : list(map(str, df_temp['weights_option'].unique())), 
    "local_restart" : list(map(str, df_temp['local_restart'].unique())), 
    "step_size_adaptation" : list(map(str, df_temp['step_size_adaptation'].unique()))
})

print(cs)
cmaes_explainer = explainer(None, 
                 cs , 
                 algname="mod-CMA",
                 dims = [5,30],#, 10, 20, 40 
                 fids = np.arange(1,25), #,5
                 iids = len( df_temp['iid'].unique()), #20 
                 reps = len( df_temp['seed'].unique()), 
                 sampling_method = "grid",  #or random
                 grid_steps_dict = {},
                 sample_size = None,  #only used with random method
                 budget = 10000, #10000
                 seed = 1,
                 verbose = True)


cmaes_explainer.load_results(data_file)

all_cors = []

df = de_explainer.df
cma_es_df = cmaes_explainer.df
for dim in de_explainer.dims:
    fid_auc_matrix = []
    fid_auc_matrix_cma = []
    for fid in de_explainer.fids:
        df_dim_fid = df[(df['dim']==dim) & (df['fid'] == fid)]

        cma_es_df_fid  = cma_es_df[(cma_es_df['dim']==dim) & (cma_es_df['fid'] == fid)]
        
        fid_auc_matrix.append(df_dim_fid.groupby(features)['auc'].mean())
        fid_auc_matrix_cma.append(cma_es_df_fid.groupby(features2)['auc'].mean())

    fid_auc_matrix = np.array(fid_auc_matrix).T
    fid_auc_matrix_cma = np.array(fid_auc_matrix_cma).T

    df_matrix = pd.DataFrame(fid_auc_matrix, columns=[f"$f_{{{i}}}$" for i in np.arange(1,25)])
    corr = df_matrix.corr()
    all_cors.append(corr)

    df_matrix2 = pd.DataFrame(fid_auc_matrix_cma, columns=[f"$f_{{{i}}}$" for i in np.arange(1,25)])
    corr2 = df_matrix2.corr()
    all_cors.append(corr2)


    if False:
        g = sns.clustermap(corr, 
                        method = 'complete', 
                        cmap   = 'viridis', 
                        annot  = True, 
                        annot_kws = {'size': 8})
        plt.tight_layout()
        plt.savefig(f"de-clustermap-{dim}d.pdf")
        plt.clf()

        g = sns.heatmap(corr, 
                        annot  = False,
                        cmap="viridis")
        plt.tight_layout()
        plt.savefig(f"de-heatmap-{dim}d.pdf")
        plt.clf()

all_cors = np.array(all_cors)

print(all_cors.shape)
min_all_cors = np.min(all_cors, axis=0)
print(min_all_cors.shape)

alldf = pd.DataFrame(min_all_cors, columns=[f"$f_{{{i}}}$" for i in np.arange(1,25)])

g = sns.clustermap(alldf, 
                method = 'complete', 
                cmap   = 'viridis', 
                annot  = True, 
                annot_kws = {'size': 8})
plt.tight_layout()
plt.savefig(f"all-clustermap.pdf")
plt.clf()
plt.clf()
g = sns.heatmap(alldf, 
                annot  = False,
                cmap="viridis")
plt.tight_layout()
plt.savefig(f"min-cor-all.pdf")
plt.clf()

Configuration space object:
  Hyperparameters:
    base_sampler, Type: Categorical, Choices: {sobol, gaussian, halton}, Default: sobol
    elitist, Type: Categorical, Choices: {True, False}, Default: True
    local_restart, Type: Categorical, Choices: {nan, IPOP, BIPOP}, Default: nan
    mirrored, Type: Categorical, Choices: {mirrored pairwise, nan, mirrored}, Default: mirrored pairwise
    step_size_adaptation, Type: Categorical, Choices: {csa, psr}, Default: csa
    weights_option, Type: Categorical, Choices: {default, equal, 1/2^lambda}, Default: default

(4, 24, 24)
(24, 24)


The figure layout has changed to tight
The figure layout has changed to tight


<Figure size 1000x1000 with 0 Axes>